# OIE

In [1]:
import pandas as pd
from nltk.tokenize import sent_tokenize
import spacy
import re

In [2]:
parser = spacy.load('en_core_web_sm')

## Input

In [3]:
with open('res/Turing.txt') as file:
    sents = sent_tokenize(file.read())

## Removing "[...]"

In [4]:
for i in range(len(sents)):
    sents[i] = re.sub(r'\[[^)]*\]', '', sents[i]).strip()

## Token expansion function

In [8]:
def expand(tokens, left = False):
    sent = []
    taboo = ["advcl", "punct", "prep", "cc", "", "compound"] if left > 0 else ["advcl", "punct", "conj", "cc", ""]
    if tokens:
        for t in tokens:
            if t.dep_ not in taboo:
                sent += expand(t.lefts,True) + [t.text] + expand(t.rights)
    return sent

## Parsing, extraction and structuring

In [10]:
triples = []
for sent in sents:
    for token in parser(sent):
        if('ROOT' in token.dep_):
            left = expand(token.lefts, True)
            right = expand(token.rights)
            if left and right:
                triples.append([" ".join(left), token.text, " ".join(right)])
df = pd.DataFrame(triples, columns = ["left", "root", "right"])

## Query

In [31]:
print(df[df['left'].str.contains("Turing")])

                       left        root  \
0                    Turing         was   
1                    Turing         was   
2          Turing is widely  considered   
4                    Turing      worked   
7                    Turing      played   
..                      ...         ...   
315        Legacy of Turing         See   
316              Turing was   appointed   
318         Turing has been    honoured   
320  A statue of Turing was    unveiled   
322               Turing is       shown   

                                                 right  
0                             an English mathematician  
1    highly influential in the development of theor...  
2              to be the father of theoretical science  
4    for the Code GC&CS at Park Britain 's codebrea...  
7    a crucial role in cracking intercepted coded m...  
..                                                 ...  
315      also List of things named after Turing Awards  
316      an officer of the 